In [3]:
from EBSD2GRID import EBSD2GRID
import numpy as np
import damask
import pandas as pd
import yaml

In [15]:
ctf_file = "Cu.ctf"
EBSD2GRID(ctf_file)

In [16]:
Fix_data = pd.read_csv("Cu.csv", sep='\t', dtype=str)

# phase = 0 的 都改成 1
# 相当于消除领解析的相

Fix_data.loc[Fix_data['phase'] == '0', 'phase'] = '1'


# phase = 1 的都改为 相名称
Fix_data.loc[Fix_data['phase'] == '1', 'phase'] = 'Cu'
#Fix_data.loc[Fix_data['phase'] == 2, 'phase'] = 'Cu2' 如果有第二相

Fix_data.to_csv("Cu.csv", index=False, sep=' ')

In [28]:
with open("Hooke_Cu.yaml", 'r', encoding='utf-8') as f:
    Cu_elasticity = yaml.safe_load(f)

with open("phenopowerlaw_Cu.yaml", 'r', encoding='utf-8') as f:
    Cu_plasticity = yaml.safe_load(f)

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'}  # No volume fractions, so no homogenization needed
    }
}

# 会多一个Cu, 删了就行
phase_Cu = {
    'Cu': {
        'lattice': 'cF',
        'rho' : 8960.0,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': Cu_elasticity,
            'plastic': Cu_plasticity
        }
    }
}

# phase_Cu = {
#         'lattice': 'cF',
#         'rho' : 8960.0,
#         'mechanical': {
#             'output': ['F', 'P', 'F_e', 'L_p', 'O'],
#             'elastic': Cu_elasticity,
#             'plastic': Cu_plasticity
#     }
# }

In [29]:
Model = damask.Table.load("Cu.csv")

gridfile = damask.GeomGrid.from_table(Model, 'pos', ['phase', 'qu'])
gridfile.origin = [0, 0, 0]
gridfile.size[2] = 0.05
CM = damask.ConfigMaterial.from_table(Model, phase='phase', O='qu', homogenization='homogenization')
CM['phase']['Cu'] = phase_Cu
CM['homogenization']= homogenization

In [25]:
CM['phase']

{'Cu': {'Cu': {'lattice': 'cF',
   'rho': 8960.0,
   'mechanical': {'output': ['F', 'P', 'F_e', 'L_p', 'O'],
    'elastic': {'type': 'Hooke',
     'references': ['W.C. Overton Jr. and J. Gaffney, Physical Review 98(4):969-977, 1955, https://doi.org/10.1103/PhysRev.98.969, fit to Tab. I (T_min=100K, T_max=300K)',
      'Y.A. Chang and L. Himmel, Journal of Applied Physics 37:3567-3572, 1966, https://doi.org/10.1063/1.1708903, fit to Tab. II (T_min=300K, T_max=800K)'],
     'C_11': 168900000000.0,
     'C_11,T': -33320000.0,
     'C_11,T^2': -10740.0,
     'C_12': 121800000000.0,
     'C_12,T': -14020000.0,
     'C_12,T^2': -9003.0,
     'C_44': 75800000000.0,
     'C_44,T': -25550000.0,
     'C_44,T^2': -2188.0,
     'T_ref': 293.15},
    'plastic': {'type': 'phenopowerlaw',
     'references': ['T. Takeuchi, Transactions of the Japan Institute of Metals 16(10):629-640, 1975, https://doi.org/10.2320/matertrans1960.16.629, fitted to Fig. 3b ([111] and [001])'],
     'output': ['xi_sl', 'g

In [27]:
gridfile.save("Cu_ebsd")
CM.save("Cu_ebsd.yaml")